In [ ]:
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve, confusion_matrix
from torchvision import datasets
from torch.utils.data import DataLoader

## Config

In [ ]:
# Constants
NUM_CLASSES = 2
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
NUM_FOLDS = 10
MODEL_SAVE_DIR = "saved_models_10"

In [ ]:

# Function to load and preprocess data
def load_data():
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    dataset = datasets.ImageFolder(root='../dataset', transform=transform)
    return dataset

# Function to define ResNet-50 model
def create_resnet_model():
    model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)
    return model

# Function to evaluate model
def evaluate_model(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    
    cm = confusion_matrix(y_true, y_pred)
    
    return accuracy, precision, recall, f1, roc_auc, pr_auc, cm, fpr, tpr

# Function to plot ROC curve
def plot_roc_curve(fpr, tpr, auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Function to plot Precision-Recall curve
def plot_precision_recall_curve(recall, precision, auc):
    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label='PR curve (area = %0.2f)' % auc)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall curve')
    plt.legend(loc="lower right")
    plt.show()



In [ ]:
# Create directory to save models
if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)

# Load data
dataset = load_data()

# Initialize k-fold cross validation
kf = KFold(n_splits=NUM_FOLDS, shuffle=True)

## Training the model

In [ ]:
# Perform k-fold cross validation
fold = 0
for train_index, test_index in kf.split(dataset):
    fold += 1
    print(f"Fold {fold}...")

    # Create model
    model = create_resnet_model()
    model = model.cuda()

    # Define data loaders
    train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
    test_sampler = torch.utils.data.SubsetRandomSampler(test_index)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

    # Define optimizer and loss function
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train model
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss}")

    # Evaluate model
    accuracy, precision, recall, f1, roc_auc, pr_auc, cm, fpr, tpr = evaluate_model(model, test_loader)

    # Save model
    torch.save(model.state_dict(), os.path.join(MODEL_SAVE_DIR, f"model_fold{fold}.pth"))

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("ROC AUC:", roc_auc)
    print("PR AUC:", pr_auc)
    print("Confusion Matrix:\n", cm)

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc)

    # Plot Precision-Recall curve
    plot_precision_recall_curve(recall, precision, pr_auc)

## Testing patients folder

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Constants
IMG_SIZE = 224
MODEL_PATH = "saved_models_10"
CLASS_NAMES = ["infected", "not_infected"]  # Replace with your class names
ROOT_FOLDER = "../CTIMGS"
# TEST_FOLDERS = ["SHAKUNTALA_NAYAK", "SULOCHANA_DAS", "person1", "BHARATI_BAG", "SARADA_KANHAR", "SAROJ_KUMAR_BEHERA", "SUCHITRA_JENA"]  # Replace with your test folders
TEST_FOLDERS = os.listdir(ROOT_FOLDER)[:10]

# Function to load the model
def load_model(m):
    model = models.resnet50(weights=None)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(CLASS_NAMES))
    model.load_state_dict(torch.load(os.path.join(MODEL_PATH, m)))
    model.eval()
    return model

# Function to preprocess image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Function to predict class for an image
def predict_image_class(image_path):
    image = preprocess_image(image_path)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    class_idx = predicted.item()
    return CLASS_NAMES[class_idx]

# Function to test a folder and print the results
def test_folder(folder_path):
    print(f"Testing folder: {folder_path}")
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Predict class for each image in the folder
    predicted_classes = []
    for image_file in image_files:
        predicted_class = predict_image_class(image_file)
        predicted_classes.append(predicted_class)

    # Count the number of predictions for each class
    infected_count = predicted_classes.count("infected")
    not_infected_count = predicted_classes.count("not_infected")
    
    print("Infected : ", infected_count, "Not infected : ", not_infected_count)
    
    # Determine the class with the higher number of predictions
    if infected_count > not_infected_count:
        print(f"The folder contains {infected_count} 'infected' images.")
    elif infected_count < not_infected_count:
        print(f"The folder contains {not_infected_count} 'not infected' images.")
    else:
        print("The folder contains an equal number of 'infected' and 'not infected' images.")
    print()

# Load the trained model onto GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
saved_models = ["model_fold1.pth", "model_fold2.pth", "model_fold3.pth", "model_fold4.pth", "model_fold5.pth", "model_fold6.pth", "model_fold7.pth", "model_fold8.pth", "model_fold9.pth", "model_fold10.pth"]
# saved_models = ["model_fold5.pth"]
for m in saved_models:
    model = load_model(m).to(device)

    # Test each folder and display the results with progress status
    for folder in TEST_FOLDERS:
        test_folder(os.path.join(ROOT_FOLDER, folder))
        print("----------------------------------------")
    print("================================================")

## Visualization using GradCAM

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
# Constants
IMG_SIZE = 224
MODEL_PATH = "saved_models/model_fold1.pth"
CLASS_NAMES = ["infected", "not_infected"]
ROOT_FOLDER = "../CTIMGS"

In [ ]:
# Function to preprocess image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image.to(device)

In [ ]:
# Load the model
def load_model():
    model = models.resnet50(weights=None)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(CLASS_NAMES))
    model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cuda:0')))
    model.eval()
    return model

In [ ]:
def generate_gradcam(image_path, model, target_class):
    # Load the image and preprocess it
    image = preprocess_image(image_path)
    
    # Forward pass
    output = model(image)
    
    # Get the gradients of the target class output with respect to the feature map
    model.zero_grad()
    target_class_score = output[:, target_class]
    target_class_score.backward()
    
    # Get the gradients from the last convolutional layer
    gradients = model.conv1.weight.grad
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    
    # Get the activations of the last convolutional layer
    activations = model.conv1(image).detach()
    
    # Weight the channels by corresponding gradients
    for i in range(pooled_gradients.shape[0]):
#         print(gradients.shape)
#         print(activations.shape)
#         print(pooled_gradients.shape)
        activations[:, i, :, :] *= pooled_gradients[i]
    
    # Average the channels of the activations
    heatmap = torch.mean(activations, dim=1).squeeze()
    
    # Normalize the heatmap
    heatmap = torch.clamp(heatmap, min=0)
    heatmap /= torch.max(heatmap)
    
    # Convert heatmap to numpy array
    heatmap = heatmap.cpu().numpy()
    
    # Resize heatmap to match the original image size
    heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))
    
    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Load the original image
    original_image = cv2.imread(image_path)
    original_image = cv2.resize(original_image, (IMG_SIZE, IMG_SIZE))
    
    # Overlay heatmap on the original image
    overlaid_image = cv2.addWeighted(original_image, 0.5, heatmap, 0.5, 0)
    
    return overlaid_image

In [ ]:
# Function to predict class for an image
def predict_image_class(image_path):
    image = preprocess_image(image_path)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    class_idx = predicted.item()
    return CLASS_NAMES[class_idx]

In [ ]:
def test_folder(folder_path):
    print(f"Testing folder: {folder_path}")
    
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for image_file in image_files:
        predicted_class = predict_image_class(image_file)
        
        # Generate Grad-CAM
        gradcam = generate_gradcam(image_file, model, CLASS_NAMES.index(predicted_class))
        output_path = os.path.splitext(image_file)[0] + "_gradcam.jpg"
        save_dir = "GradCAM/"+output_path.split('/')[2]+'/'+output_path.split('/')[3]
        cv2.imwrite(save_dir, gradcam)
        print(save_dir, "saved")

In [ ]:
# Load the model onto GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model().to(device)

In [ ]:
TEST_FOLDERS = ["person1", "person2"]  # Replace with your test folders

# Test each folder and display the results with Grad-CAMs
for folder in TEST_FOLDERS:
    try:
        os.mkdir("GradCAM/"+folder)
    except:
        print("File exists")
    test_folder(os.path.join(ROOT_FOLDER, folder))

### generating some visualizations

In [ ]:
import os
from PIL import Image

def combine_images(image_folder1, image_folder2, output_folder):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # List image files in both folders
    images1 = sorted(os.listdir(image_folder1))
    images2 = sorted(os.listdir(image_folder2))

    # Iterate through images and combine them
    for img1_name, img2_name in zip(images1, images2):
        if img1_name.endswith(".jpg") and img2_name.endswith(".jpg"):
            img1_path = os.path.join(image_folder1, img1_name)
            img2_path = os.path.join(image_folder2, img2_name)
            img1 = Image.open(img1_path)
            img2 = Image.open(img2_path)
            
            # Combine images horizontally
            combined_img = Image.new('RGB', (img1.width + img2.width, img1.height))
            combined_img.paste(img1, (0, 0))
            combined_img.paste(img2, (img1.width, 0))

            # Save combined image to output folder
            combined_img.save(os.path.join(output_folder, f"{img1_name[:-4]}_combined.jpg"))

# Example usage:
image_folder1 = "../CTIMGS/person1"
image_folder2 = "GradCAM/person1"
output_folder = "combined_images_person1"
combine_images(image_folder1, image_folder2, output_folder)